In [ ]:
import pandas as pd

# Load your data into a Pandas DataFrame
df = pd.read_excel('Classpass work.xlsx')

# Drop duplicates based on 'Answer.classMapURL'
df_cleaned = df.drop_duplicates(subset='Answer.classMapURL', keep='first')

# Additional cleaning steps if needed
# For example, you might want to handle missing values
df_cleaned.dropna(inplace=True)

# Save the cleaned data to a new file
df_cleaned.to_excel('cleaned_data.xlsx', index=False)


In [ ]:
# Load Excel data into a Pandas DataFrame
df = pd.read_excel('cleaned_data.xlsx')

# Display the DataFrame
print(df)

                                 Answer.classDuration  \
0                                              7 days   
1                                             14 Days   
2                                               1 day   
3                                             I class   
4                                              3 days   
5                                                  15   
6                                             10 days   
7                                                   3   
8                                                  14   
9                           7 days- unlimited classes   
10                                 2 CLASSES PER WEEK   
11                                             3 days   
12                                              1 day   
13                                         1 year ago   
14                                            15 days   
15                                                  7   
16                             

In [ ]:
pip install geopy


In [ ]:
import pandas as pd
import re

# Load your data into a Pandas DataFrame
df = pd.read_excel('cleaned_data.xlsx')

# Regular expression pattern to extract coordinates from Google Maps URLs
pattern = r'@([-+]?\d+\.\d+),([-+]?\d+\.\d+),\d+z'

# Function to extract coordinates from Google Maps URL
def extract_coordinates(url):
    match = re.search(pattern, url)
    if match:
        return float(match.group(1)), float(match.group(2))
    else:
        return None, None

# Apply the function to the 'Answer.classMapURL' column
df['latitude'], df['longitude'] = zip(*df['Answer.classMapURL'].apply(extract_coordinates))

# Print the DataFrame with added latitude and longitude columns
print(df)

# Save the DataFrame with extracted coordinates to a new file
df.to_excel('geocoded_data_from_url.xlsx', index=False)


In [ ]:
pip install pandas requests


In [ ]:
import pandas as pd
import re
import requests
from urllib.parse import urlparse

# Load your data into a Pandas DataFrame
df = pd.read_excel('geocoded_data_from_url.xlsx')

# Function to extract coordinates from Google Maps short URL
def extract_coordinates_from_short_url(short_url):
    # Check if the URL has a valid scheme; if not, add 'http://' as the default scheme
    parsed_url = urlparse(short_url)
    if not parsed_url.scheme:
        short_url = 'http://' + short_url

    try:
        # Fetch the long URL from the short URL
        response = requests.head(short_url, allow_redirects=True)
        long_url = response.url

        # Extract latitude and longitude from the long URL
        pattern = r'@([-+]?\d+\.\d+),([-+]?\d+\.\d+),\d+z'
        match = re.search(pattern, long_url)

        if match:
            latitude, longitude = float(match.group(1)), float(match.group(2))
            return latitude, longitude
        else:
            return None, None
    except requests.RequestException:
        # Handle the case where the URL is not accessible
        return None, None

# Apply the function to the 'Answer.classMapURL' column
df['latitude'], df['longitude'] = zip(*df['Answer.classMapURL'].apply(extract_coordinates_from_short_url))

# Print the DataFrame with added latitude and longitude columns
print(df)

# Save the DataFrame with extracted coordinates to a new file
df.to_excel('extracted_coordinates_data.xlsx', index=False)


                                 Answer.classDuration  \
0                                              7 days   
1                                             14 Days   
2                                               1 day   
3                                             I class   
4                                              3 days   
5                                                  15   
6                                             10 days   
7                                                   3   
8                                                  14   
9                           7 days- unlimited classes   
10                                 2 CLASSES PER WEEK   
11                                             3 days   
12                                              1 day   
13                                         1 year ago   
14                                            15 days   
15                                                  7   
16                             

In [ ]:
import pandas as pd
import requests

# Load your data into a Pandas DataFrame
df = pd.read_excel('extracted_coordinates_data.xlsx')

# Function to reverse geocode and extract city names, titles, and descriptions
def reverse_geocode(latitude, longitude):
    try:
        # Perform reverse geocoding using Nominatim
        url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}&zoom=10'
        response = requests.get(url)
        data = response.json()

        # Extract city name
        city = data.get('address', {}).get('city', 'N/A')

        return city
    except Exception as e:
        print(f"Error: {e}")
        return 'N/A'

# Apply the function to the latitude and longitude columns
df['city'] = df.apply(lambda row: reverse_geocode(row['latitude'], row['longitude']), axis=1)

# Print the DataFrame with added city column
print(df)

# Save the DataFrame with extracted city names to a new file
df.to_excel('extracted_city_data.xlsx', index=False)


In [ ]:
import pandas as pd
import requests

# Load your data into Pandas DataFrames
df_cities = pd.read_excel('extracted_city_data.xlsx')

# Function to reverse geocode and extract titles
def reverse_geocode_for_title(latitude, longitude):
    try:
        # Perform reverse geocoding using Nominatim
        url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}&zoom=10'
        response = requests.get(url)
        data = response.json()

        # Extract place title
        title = data.get('display_name', 'N/A')

        return title
    except Exception as e:
        print(f"Error: {e}")
        return 'N/A'

# Apply the function to the latitude and longitude columns in df_cities
df_cities['title'] = df_cities.apply(lambda row: reverse_geocode_for_title(row['latitude'], row['longitude']), axis=1)

# Print the DataFrame with added title column
print(df_cities)

# Save the DataFrame with extracted city, title, and description data to a new file
df_cities.to_excel('extracted_city_title_data.xlsx', index=False)


                                 Answer.classDuration  \
0                                              7 days   
1                                             14 Days   
2                                               1 day   
3                                             I class   
4                                              3 days   
5                                                   3   
6                                                  14   
7                           7 days- unlimited classes   
8                                  2 CLASSES PER WEEK   
9                                              3 days   
10                                              1 day   
11                                         1 year ago   
12                                                  7   
13                                             5 days   
14                                                 30   
15                                              1 day   
16                             

In [ ]:
import pandas as pd
import requests

# Load your data into a Pandas DataFrame
df_combined = pd.read_excel('extracted_city_title_data.xlsx')

# Function to reverse geocode and extract titles
def reverse_geocode_for_title(latitude, longitude):
    try:
        # Perform reverse geocoding using Nominatim
        url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}&zoom=10'
        response = requests.get(url)
        data = response.json()

        # Extract place title
        title = data.get('display_name', 'N/A')

        return title
    except Exception as e:
        print(f"Error: {e}")
        return 'N/A'

# Function to extract the part of the title before the first arrow
def extract_city_from_title(title):
    if ' > ' in title:
        return title.split(' > ', 1)[0]
    else:
        return title

# Apply the function to the latitude and longitude columns in df_combined
df_combined['title'] = df_combined.apply(lambda row: reverse_geocode_for_title(row['latitude'], row['longitude']), axis=1)

# Extract city from the title and update the 'city' column
df_combined['city'] = df_combined['title'].apply(extract_city_from_title)

# Print the DataFrame with added title and updated city columns
print(df_combined)

# Save the DataFrame with updated city column to the same file
df_combined.to_excel('extracted_city_title_data.xlsx', index=False)


                                 Answer.classDuration  \
0                                              7 days   
1                                             14 Days   
2                                               1 day   
3                                             I class   
4                                              3 days   
5                                                   3   
6                                                  14   
7                           7 days- unlimited classes   
8                                  2 CLASSES PER WEEK   
9                                              3 days   
10                                              1 day   
11                                         1 year ago   
12                                                  7   
13                                             5 days   
14                                                 30   
15                                              1 day   
16                             

In [ ]:
import pandas as pd

# Load your data into a Pandas DataFrame
df_combined = pd.read_excel('extracted_city_county_state_data.xlsx')

# Print the DataFrame before dropping the 'title' column
print("Before dropping 'title' column:")
print(df_combined)

# Drop the original "title" column
df_combined = df_combined.drop(columns=['Title'])

# Print the DataFrame after dropping the 'title' column
print("\nAfter dropping 'title' column:")
print(df_combined)

# Save the DataFrame to the same file without the 'title' column
df_combined.to_excel('extracted_city_county_state_data.xlsx', index=False)


Before dropping 'title' column:
                                 Answer.classDuration  \
0                                              7 days   
1                                             14 Days   
2                                               1 day   
3                                             I class   
4                                              3 days   
5                                                   3   
6                                                  14   
7                           7 days- unlimited classes   
8                                  2 CLASSES PER WEEK   
9                                              3 days   
10                                              1 day   
11                                         1 year ago   
12                                                  7   
13                                             5 days   
14                                                 30   
15                                              1 day   

KeyError: ignored